# 1. Knowledge Distillation and Hint Layers on CIFAR-10
This notebook contains the code for two experiments on CIFAR-10. The text accompanying the code is brief, as most of the explanation is in the main pdf-file.

First, the packages are imported.

In [1]:
from __future__ import print_function
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input, Lambda, concatenate
from tensorflow.keras.losses import categorical_crossentropy as logloss
import numpy as np

2.2.0


In the code below the CIFAR-10 data is imported and normalised.

In [3]:
nb_classes = 10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# convert y_train and y_test to categorical binary values 
Y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
Y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

X_train = X_train.reshape(50000, 32,32,3)
X_test = X_test.reshape(10000, 32,32,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize the values
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

input_shape = (32,32,3) # Input shape of each image

170500096/170498071 [==============================] - 11s 0us/step
50000 train samples
10000 test samples


In the code block below a CNN with an architecture similar to VGG is created and explained in its output.

In [4]:
inputs = Input(shape=input_shape)

t_conv1 = Conv2D(64, kernel_size=(3, 3),
                 activation='relu', 
                 padding = 'same', 
                 kernel_initializer='he_normal')(inputs)
t_conv2 = Conv2D(64, (3, 3), activation='relu', padding = 'same')(t_conv1)
t_maxpool1 = MaxPooling2D(pool_size=(2, 2))(t_conv2)

# No dropout added here, as the hint layer is only created later, and adding dropout in between
# might not be smart

t_conv3 = Conv2D(128, kernel_size=(3, 3),
                 activation='relu',
                 padding = 'same')(t_maxpool1)
t_conv4 = Conv2D(128, (3, 3), activation='relu',padding='same')(t_conv3)
t_maxpool2 = MaxPooling2D(pool_size=(2, 2))(t_conv4)


t_conv5 = Conv2D(256, kernel_size=(3, 3),
                 activation='relu',
                 padding = 'same')(t_maxpool2)
t_conv6 = Conv2D(256, (3, 3), activation='relu', padding = 'same')(t_conv5)
t_conv7 = Conv2D(256, (3, 3), activation='relu', padding = 'same')(t_conv6)

# Create the hint 'model'
w_hint = Model(inputs=inputs, outputs = t_conv7, name = 'w_hint')

t_maxpool3 = MaxPooling2D(pool_size=(2, 2))(t_conv7)

t_do1 = Dropout(0.4)(t_maxpool3)

t_conv8 = Conv2D(512, kernel_size=(3, 3),
                 activation='relu',
                 padding = 'same')(t_do1)
t_conv9 = Conv2D(512, (3, 3), activation='relu', padding = 'same')(t_conv8)
t_conv10 = Conv2D(512, (3, 3), activation='relu', padding = 'same')(t_conv9)
t_conv11 = Conv2D(512, (3, 3), activation='relu', padding = 'same')(t_conv10)
t_maxpool4 = MaxPooling2D(pool_size=(2, 2))(t_conv11)

t_do2 = Dropout(0.4)(t_maxpool4)

t_flat = Flatten()(t_do2)
t_dense1 = Dense(1024, activation='relu')(t_flat)
t_dense2 = Dense(256, activation='relu')(t_dense1)
t_dense3 = Dense(64, activation='relu')(t_dense2)

t_do3 = Dropout(0.3)(t_dense3)

t_dense_final = Dense(nb_classes, name = 'wo_softmax_teach')(t_do3)
t_softmax = Activation('softmax')(t_dense_final)
# Note that we add a normal softmax layer to begin with
teacher = Model(inputs=inputs, outputs=t_softmax, name = 'teacher')


# momentum optimiser
t_optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.9, nesterov=True, name='SGD'
)

teacher.compile(loss='categorical_crossentropy',
              optimizer=t_optimizer,
              metrics=['accuracy'])
print(teacher.summary())

Model: "teacher"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0   

In [ ]:
# Training the teacher
epochs = 40
batch_size = 128
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

Epoch 1/40
391/391 [==============================] - 12s 32ms/step - loss: 2.2947 - accuracy: 0.1109 - val_loss: 2.3021 - val_accuracy: 0.1030
Epoch 2/40
391/391 [==============================] - 12s 30ms/step - loss: 2.2632 - accuracy: 0.1393 - val_loss: 2.0640 - val_accuracy: 0.2575
Epoch 3/40
391/391 [==============================] - 12s 30ms/step - loss: 1.9529 - accuracy: 0.2666 - val_loss: 1.7121 - val_accuracy: 0.3575
Epoch 4/40
391/391 [==============================] - 12s 30ms/step - loss: 1.6594 - accuracy: 0.3724 - val_loss: 1.4987 - val_accuracy: 0.4251
Epoch 5/40
391/391 [==============================] - 12s 30ms/step - loss: 1.4129 - accuracy: 0.4811 - val_loss: 1.2276 - val_accuracy: 0.5484
Epoch 6/40
391/391 [==============================] - 12s 30ms/step - loss: 1.2063 - accuracy: 0.5685 - val_loss: 1.2954 - val_accuracy: 0.5445
Epoch 7/40
391/391 [==============================] - 12s 30ms/step - loss: 1.0295 - accuracy: 0.6352 - val_loss: 0.9717 - val_accuracy:

In the code below the student network is created.

In [5]:
inputs = Input(shape=input_shape)

s_conv1 = Conv2D(16, kernel_size=(3, 3),
                 activation='relu', 
                 padding = 'same', 
                 kernel_initializer='he_normal')(inputs)
s_maxpool1 = MaxPooling2D(pool_size=(2, 2))(s_conv1)

s_conv2 = Conv2D(32, (3, 3), activation='relu',padding='same')(s_maxpool1)
s_maxpool2 = MaxPooling2D(pool_size=(2, 2))(s_conv2)

# Guided 'model'
w_guided =  Model(inputs=inputs, outputs=s_maxpool2, name = 'w_guided')

s_flat1 = Flatten()(s_maxpool2)
s_dense1 = Dense(128, activation='relu')(s_flat1)

s_do1 = Dropout(0.3)(s_dense1)

s_dense_final = Dense(nb_classes, name = 'wo_softmax_stud')(s_do1)
s_softmax = Activation('softmax')(s_dense_final)

student = Model(inputs=inputs, outputs=s_softmax, name = 'student')

student.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

student.summary()

Model: "student"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               2622

In [ ]:
# Train the student to achieve base accuracy
epochs = 30
batch_size = 64
student.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))
# 0.6932, 0.7047, 0.7026, 0.6953

Epoch 1/30
782/782 [==============================] - 3s 4ms/step - loss: 1.6039 - accuracy: 0.4158 - val_loss: 1.2752 - val_accuracy: 0.5387
Epoch 2/30
782/782 [==============================] - 3s 4ms/step - loss: 1.2949 - accuracy: 0.5373 - val_loss: 1.1566 - val_accuracy: 0.5855
Epoch 3/30
782/782 [==============================] - 3s 4ms/step - loss: 1.1706 - accuracy: 0.5855 - val_loss: 1.0758 - val_accuracy: 0.6209
Epoch 4/30
782/782 [==============================] - 3s 4ms/step - loss: 1.0901 - accuracy: 0.6128 - val_loss: 1.0384 - val_accuracy: 0.6347
Epoch 5/30
782/782 [==============================] - 3s 4ms/step - loss: 1.0327 - accuracy: 0.6343 - val_loss: 0.9789 - val_accuracy: 0.6530
Epoch 6/30
782/782 [==============================] - 3s 4ms/step - loss: 0.9870 - accuracy: 0.6506 - val_loss: 0.9611 - val_accuracy: 0.6633
Epoch 7/30
782/782 [==============================] - 3s 4ms/step - loss: 0.9377 - accuracy: 0.6691 - val_loss: 0.9439 - val_accuracy: 0.6635
Epoch 

In [ ]:
# Delete the model so that it can be retrained from scratch
# Again, to achieve base accuracy.
student.reset_states()
student = None
del student

### Hint and guided layer code

In the code below the adaptation layer is created, and Wr is created as described in the paper. It is checked if the output of both the hint layer and the guided layer + adaptation layer is the same.

In [ ]:
# conv_regressor is the adapation layer.
conv_regressor = Conv2D(256, kernel_size=(1, 1),
                 input_shape = (8,8,32),
                 kernel_initializer='glorot_normal')(w_guided.output)

w_r = Model(inputs=inputs, outputs=conv_regressor, name = 'w_r')
print(w_hint.output)
print(w_r.output)
print(teacher.output)

Tensor("conv2d_6/Identity:0", shape=(None, 8, 8, 256), dtype=float32)
Tensor("conv2d_13/Identity:0", shape=(None, 8, 8, 256), dtype=float32)
Tensor("activation/Identity:0", shape=(None, 10), dtype=float32)


Creating the funciton below took me very long to figure out. In essence it takes the l2 distance between the guided layer and hint layer output.

In [ ]:
def fitnet_loss(target_feat, source_feat):
    return tf.reduce_mean(tf.square(target_feat-source_feat))

In [ ]:
w_r.compile(
    optimizer='adam',
    loss= (lambda y_hint, y_guided: fitnet_loss(y_hint, y_guided)))
w_hint_outputs = w_hint.predict(X_train)
print(w_hint_outputs.shape)


(50000, 8, 8, 256)


The hint outputs are then created, and the w_r trains the layers until the guided layer from the student model by taking the l2 distance between the hint output and the guided + adaptation layer output. Minimum loss is achieved within 9 epochs.

In [ ]:
w_r.fit(X_train, w_hint_outputs,
          batch_size=128,
          epochs=10)#,
          #verbose=1)

Epoch 1/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0226
Epoch 2/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0218
Epoch 3/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0216
Epoch 4/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0215
Epoch 5/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0215
Epoch 6/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0214
Epoch 7/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0214
Epoch 8/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0213
Epoch 9/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0212
Epoch 10/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0212


### Knowledge distillation code

In the code below a manual softmax function is defined, and a new teacher model without its softmax output is created. 

In [ ]:
# Define a manual softmax function
def softmax(x):
    return np.exp(x)/(np.exp(x).sum())

teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('wo_softmax_teach').output)


In [ ]:
teacher_train_logits = teacher_WO_Softmax.predict(X_train)
teacher_test_logits = teacher_WO_Softmax.predict(X_test) 
# This model directly gives the soft logits 

This output is then saved because otherwise I would have to retrain the teacher model each time I would want to train the student model with knowledge distillation.

In [ ]:
teacher_train_logits = np.load("teacher_train_logits.npy")
teacher_test_logits = np.load("teacher_test_logits.npy")

The code below is rerun for all temperature values (2, 4, 6, 8, 10, 12 and 14).

In [ ]:
# Set a tempature value
temp = 12


# Perform a manual softmax at raised temperature
train_logits_T = teacher_train_logits / temp
test_logits_T = teacher_test_logits / temp 

Y_train_soft = softmax(train_logits_T)
Y_test_soft = softmax(test_logits_T)

# Concatenate so that this becomes a 10 + 10 dimensional vector
Y_train_new = np.concatenate([Y_train, Y_train_soft], axis=1)
Y_test_new =  np.concatenate([Y_test, Y_test_soft], axis =1)

In [ ]:
print(Y_train_new.shape)
print(Y_test_new.shape)
print(X_train.shape)
Y_train_new[1]
print(type(teacher_train_logits))

(50000, 20)
(10000, 20)
(50000, 32, 32, 3)
<class 'numpy.ndarray'>


Below the student model with a concatenation of two outputs is created, as described in the paper and Hinton's original knowledge distillation paper.

In [7]:
# Now collect the logits from the last layer
logits = student.layers[-2].output # This is going to be a tensor. And hence it needs to pass through a Activation layer
probs = Activation('softmax')(logits)

# softened probabilities at raised temperature
logits_T = tf.keras.layers.Lambda(lambda x: x / temp)(logits)
probs_T = Activation('softmax')(logits_T)

output = tf.keras.layers.concatenate([probs, probs_T])

# This is the new student model
student_kd = Model(student.input, output)

student_kd.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 16, 16, 16)   0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 16, 16, 32)   4640        max_pooling2d_4[0][0]            
______________________________________________________________________________________________

Below the loss functions for knowledge distillation are defined.

In [ ]:
# Declare knowledge distillation loss
def knowledge_distillation_loss(y_true, y_pred, alpha):

    # Extract the one-hot encoded values and the softs separately so that we can create two objective functions
    y_true, y_true_softs = y_true[: , :nb_classes], y_true[: , nb_classes:]
    
    y_pred, y_pred_softs = y_pred[: , :nb_classes], y_pred[: , nb_classes:]
    
    loss = (1-alpha)*logloss(y_true,y_pred) + alpha*logloss(y_true_softs, y_pred_softs)
    
    return loss

# For testing use regular output probabilities - without temperature
def acc(y_true, y_pred):
    y_true = y_true[:, :nb_classes]
    y_pred = y_pred[:, :nb_classes]
    return tf.keras.metrics.categorical_accuracy(y_true, y_pred)

The codeblock below is run for all different values of alpha (0.3, 0.5, 0.7 and 0.9).

In [ ]:
student_kd.compile(
    optimizer='adam',
    loss=(lambda y_true, y_pred: knowledge_distillation_loss(y_true, y_pred, 0.5)),
    metrics=[acc] )


In [ ]:
student_kd.fit(X_train, Y_train_new,
          batch_size=64,
          epochs=30,
          verbose=1,
          validation_data=(X_test, Y_test_new))

Epoch 1/30
782/782 [==============================] - 3s 4ms/step - loss: 0.7187 - acc: 0.4791 - val_loss: 0.6095 - val_acc: 0.5684
Epoch 2/30
782/782 [==============================] - 3s 4ms/step - loss: 0.5841 - acc: 0.5827 - val_loss: 0.5585 - val_acc: 0.6002
Epoch 3/30
782/782 [==============================] - 3s 4ms/step - loss: 0.5289 - acc: 0.6247 - val_loss: 0.5065 - val_acc: 0.6437
Epoch 4/30
782/782 [==============================] - 3s 4ms/step - loss: 0.4953 - acc: 0.6517 - val_loss: 0.4784 - val_acc: 0.6647
Epoch 5/30
782/782 [==============================] - 3s 4ms/step - loss: 0.4699 - acc: 0.6675 - val_loss: 0.4753 - val_acc: 0.6678
Epoch 6/30
782/782 [==============================] - 3s 4ms/step - loss: 0.4498 - acc: 0.6813 - val_loss: 0.4723 - val_acc: 0.6701
Epoch 7/30
782/782 [==============================] - 3s 4ms/step - loss: 0.4307 - acc: 0.6961 - val_loss: 0.4572 - val_acc: 0.6794
Epoch 8/30
782/782 [==============================] - 3s 4ms/step - loss: 0.

The models had to be trained tens of times, thus the code blocks below were used to reset the weights. Both had to be run in order to recreate an initialised student_kd model.

In [ ]:
student_kd.reset_states()
student_kd = None
del student_kd
tf.keras.backend.clear_session() 

In [ ]:
student.reset_states()
student = None
del student

### Teacher-assistant
The code block below defines the student model for the multi-step knowledge distillation. The student here is called the pupil. As the results were unsatisfactory, I removed the hint layer code. The process of creating this code is equivalent precisely to what is done in the code above.

### Dense layers
I also removed the dense layer code, as these also led to unsatisfactory results. As described in the paper, the final test accuracy would always oscillate between two different values, whether they were trained on soft outputs or not.

In [ ]:
inputs = Input(shape=input_shape)


p_conv1 = Conv2D(16, kernel_size=(3, 3),
                 activation='relu', 
                 padding = 'same', 
                 kernel_initializer='he_normal')(inputs)
p_maxpool1 = MaxPooling2D(pool_size=(2, 2))(p_conv1)

p_flat1 = Flatten()(p_maxpool1)
p_dense1 = Dense(32, activation='relu')(p_flat1)
p_do1 = Dropout(0.2)(p_dense1)
p_dense_final = Dense(nb_classes, name = 'wo_softmax_pupil')(p_do1)
p_softmax = Activation('softmax')(p_dense_final)

pupil = Model(inputs=inputs, outputs=p_softmax, name = 'pupil')

pupil.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

pupil.summary()

Model: "pupil"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                131104    
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
wo_softmax_pupil (Dense)     (None, 10)                330   

In [ ]:
pupil.fit(X_train, Y_train,
          batch_size=64,
          epochs=20,
          verbose=1,
          validation_data=(X_test, Y_test))

In [ ]:
student_WO_Softmax = Model(student.input, student.get_layer('wo_softmax_').output)

student_train_logits = student_WO_Softmax.predict(X_train)
student_test_logits = student_WO_Softmax.predict(X_test)

In [ ]:
# Set a tempature value
temp = 8


# Perform a manual softmax at raised temperature
student_train_logits_T = student_train_logits / temp
student_test_logits_T = student_test_logits / temp 

Y_train_soft_student = softmax(student_train_logits_T)
Y_test_soft_student = softmax(student_test_logits_T)

# Concatenate so that this becomes a 10 + 10 dimensional vector
Y_train_new_student = np.concatenate([Y_train, Y_train_soft_student], axis=1)
Y_test_new_student =  np.concatenate([Y_test, Y_test_soft_student], axis =1)

# Now collect the logits from the last layer
logits_pupil = pupil.layers[-2].output # This is going to be a tensor. And hence it needs to pass through a Activation layer
probs_pupil = Activation('softmax')(logits_pupil)

# softened probabilities at raised temperature
logits_T_pupil = tf.keras.layers.Lambda(lambda x: x / temp)(logits_pupil)
probs_T_pupil = Activation('softmax')(logits_T_pupil)

output_pupil = tf.keras.layers.concatenate([probs_pupil, probs_T_pupil])

# This is our new student model
pupil_kd = Model(pupil.input, output_pupil)


p_optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.9, nesterov=True, name='SGD'
)


pupil_kd.compile(
    #optimizer=tf.keras.optimizers.SGD(lr=1e-1, momentum=0.9, nesterov=True),
    optimizer='adam',
    loss=(lambda y_true, y_pred: knowledge_distillation_loss(y_true, y_pred, 0.5)),
    #loss='categorical_crossentropy',
    metrics=[acc] )


pupil_kd.fit(X_train, Y_train_new_student,
          batch_size=64,
          epochs=20,
          verbose=1,
          validation_data=(X_test, Y_test_new_student))

Epoch 1/20
782/782 [==============================] - 4s 5ms/step - loss: 1.0592 - acc: 0.2165 - val_loss: 0.9253 - val_acc: 0.3236
Epoch 2/20
782/782 [==============================] - 4s 5ms/step - loss: 0.9504 - acc: 0.2835 - val_loss: 0.8609 - val_acc: 0.3848
Epoch 3/20
782/782 [==============================] - 4s 5ms/step - loss: 0.9176 - acc: 0.3042 - val_loss: 0.8358 - val_acc: 0.3714
Epoch 4/20
782/782 [==============================] - 4s 5ms/step - loss: 0.9004 - acc: 0.3145 - val_loss: 0.8244 - val_acc: 0.3899
Epoch 5/20
782/782 [==============================] - 4s 5ms/step - loss: 0.8889 - acc: 0.3200 - val_loss: 0.8219 - val_acc: 0.3787
Epoch 6/20
782/782 [==============================] - 4s 5ms/step - loss: 0.8813 - acc: 0.3209 - val_loss: 0.7919 - val_acc: 0.4005
Epoch 7/20
782/782 [==============================] - 4s 5ms/step - loss: 0.8734 - acc: 0.3238 - val_loss: 0.7907 - val_acc: 0.3961
Epoch 8/20
782/782 [==============================] - 4s 5ms/step - loss: 0.

In [ ]:
pupil.reset_states()
pupil = None
del pupil

In [ ]:
pupil_kd.reset_states()
pupil_kd = None
del pupil_kd